In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
import time

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices(device_type = None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]

In [4]:
f16 = pd.read_csv('data/f16_l3.csv')

In [5]:
X = f16['Force']
y = f16['Acceleration3']

In [6]:
#X = MinMaxScaler().fit_transform(X.values.reshape(-1,1))

In [7]:
#Declare the number of model iterations we will run 
test_iterations = 250

#Declare the size of the initial train test split
test_size = 0.20

#Declare the number of hidden layers to add
layers = 2

#Declare the density of the hidden layers
density = 128

#Declare the type of activation for the hidden layers
activation = tf.keras.layers.LeakyReLU(alpha = 0.03)

#Assign the loss function the model will use to train
loss = 'mean_squared_error'

#Assign the optimizer for the compilitation
optimizer = 'adam'

#Declare the patience count used for early stopping
patience = 25

#Declare the test size used for early stopping train test split
es_test_size = 0.25

#Declare the batch size for use in the model
batch_size = 64

#Declare the maximum number of epochs for our model
epochs = 250

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, random_state = 8669, test_size = test_size)

In [9]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    model = Sequential()

    #Creates a for loop that will add the number of layers based on the variable declared in the previous cell    
    for i in range(layers):
        #Adds a model layer with density and activation based on the variables declared in the previous cell
        model.add(Dense(density, activation = activation))
    
    model.add(Dense(1, activation='linear'))
    
    #model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

In [10]:
es = EarlyStopping(monitor = 'loss', patience = patience, restore_best_weights = True)
mc = ModelCheckpoint(filepath = 'test_model.h5', monitor = 'loss', save_best_only=True)
X_es_train, X_es_true, y_es_train, y_es_true = train_test_split(X_train, y_train, test_size = es_test_size, random_state = 8669)

In [11]:
#Creates a dataframe by which we will eventually put in our list created above
model_record = pd.DataFrame(columns = ['model_num', 'loss_type', 'time', 'epochs', 'r2', 'mae', 'mse', 'rmse', 'max_error'])

#Creates a dataframe by which our model's predicted values and true values will be stored
predict_record = pd.DataFrame(y_true).reset_index(drop = True)

#Creates a numpy array by which the for loop will use to count model runs and is then used to name df columns
model_counter = np.array([0])

In [12]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    
#Performs model training repeatedly based on the variable declared previously    
    for i in range(test_iterations):
        model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

        #Creates an empty list for storing model metrics and other information 
        record_list = list() 
        #Starts a counter that adds 1 everytime a fitting is performed
        model_counter = model_counter + 1
        #Starts a timer to end after the round of fitting is complete
        start_time = datetime.now()
        
        #Fits our model/batch_size and epochs are declared previously
        #model.fit(x = X_train, y = y_train.values, 
        #          batch_size = batch_size, epochs = epochs)
        
        history = model.fit(x = X_es_train, y = y_es_train.values, 
                  validation_data = (X_es_true, y_es_true),
                  batch_size = batch_size, epochs = epochs, 
                  callbacks = [es, mc])
        
        #Saves our model predictions
        y_pred = model.predict(X_true)
        
        #Saves our model run #, time to run, and model metrics to our temporary list
        record_list.extend([len(model_record)+1, #A model iteration count
                            loss, #Records the loss metric used to fit the model
                            format(datetime.now() - start_time), #Calculates the time it took to complete the model
                            len(history.history['loss']), #The epoch number of our model checkpoint 
                            r2_score(y_true, y_pred), #R2
                            mean_absolute_error(y_true, y_pred), #MAE
                            mean_squared_error(y_true, y_pred), #MSE
                            np.sqrt(mean_squared_error(y_true, y_pred)), #RMSE
                            max_error(y_true, y_pred) #MaxError
                           ])        
        
        #Adds the temporary list of model metrics (etc) to the end of a dataframe
        model_record.loc[len(model_record)] = record_list
        
        #Converts our predictions to a dataframe so it will play nice
        y_pred_df = pd.DataFrame(y_pred)
        
        #Adds predictions as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, y_pred_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'm' + str(model_counter[0])})
        
        #Calculates the residual values for each prediction and stores it as a dataframe 
        residuals_df = pd.DataFrame(abs(predict_record.iloc[:,len(predict_record.columns)-1] - predict_record.iloc[:,0]))
        
        #Adds y residuals as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, residuals_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'res' + str(model_counter[0])})

Epoch 1/250
1006/1006 [==============================] - 4s 3ms/step - loss: 0.2495 - mse: 0.2495 - val_loss: 0.2735 - val_mse: 0.2735
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2381 - mse: 0.2381 - val_loss: 0.2404 - val_mse: 0.2404
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2339 - mse: 0.2339 - val_loss: 0.2394 - val_mse: 0.2394
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2313 - mse: 0.2313 - val_loss: 0.2378 - val_mse: 0.2378
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2291 - mse: 0.2291 - val_loss: 0.2371 - val_mse: 0.2371
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2281 - mse: 0.2281 - val_loss: 0.2372 - val_mse: 0.2372
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2268 - mse: 0.2268 - val_loss: 0.2414 - val_mse: 0.2414
Epoch 8/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2100 - mse: 0.2100 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 62/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2101 - mse: 0.2101 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 63/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2100 - mse: 0.2100 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 64/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2098 - mse: 0.2098 - val_loss: 0.2194 - val_mse: 0.2194
Epoch 65/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2098 - mse: 0.2098 - val_loss: 0.2184 - val_mse: 0.2184
Epoch 66/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2100 - mse: 0.2100 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 67/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2100 - mse: 0.2100 - val_loss: 0.2204 - val_mse: 0.2204
Epoch 68/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 122/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 123/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 124/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2231 - val_mse: 0.2231
Epoch 125/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 126/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 127/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2232 - val_mse: 0.2232
Epoch 128/250
1006/1006 [==============================

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 13/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2094 - mse: 0.2094 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 14/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 15/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2090 - mse: 0.2090 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 16/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 17/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2207 - val_mse: 0.2207
Epoch 18/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2179 - val_mse: 0.2179
Epoch 19/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2207 - val_mse: 0.2207
Epoch 32/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 33/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2089 - mse: 0.2089 - val_loss: 0.2182 - val_mse: 0.2182
Epoch 34/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2180 - val_mse: 0.2180
Epoch 35/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2196 - val_mse: 0.2196
Epoch 36/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2184 - val_mse: 0.2184
Epoch 37/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2087 - mse: 0.2087 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 38/250
1006/1006 [==============================] - 3s 

...

1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 4/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 5/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 6/250
1006/1006 [==============================] - 3s 2ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 7/250
1006/1006 [==============================] - 3s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 8/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 9/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 10/250
1006/1006 [==============================] - 3s 2ms/st

1006/1006 [==============================] - 2s 2ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 64/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 65/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 66/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 67/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 68/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 69/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 1/250
1006/1006 [==============================] - 3s 3

1006/1006 [==============================] - 2s 2ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 5/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 6/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 7/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2075 - mse: 0.2075 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 8/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 9/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 10/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 11/250
1006/1006 [==============================] - 2s 2ms/s

1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 7/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 8/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 9/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2166 - val_mse: 0.2166
Epoch 10/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 11/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 12/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 13/250
1006/1006 [==============================] - 2s 2ms

1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 29/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2073 - mse: 0.2073 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 1/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 2/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 3/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2072 - mse: 0.2072 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 4/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2171 - val_mse: 0.2171
Epoch 5/250
1006/1006 [==============================] - 2s 2ms/step - loss: 0.2074 - mse: 0.2074 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 6/250
1006/1006 [==============================] - 2s 2ms/st

In [13]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:06:58.600213,133,0.186559,0.231756,0.217424,0.466287,5.227035
1,2,mean_squared_error,0:01:52.498483,36,0.181425,0.232965,0.218796,0.467756,5.223996
2,3,mean_squared_error,0:02:09.278538,41,0.188748,0.231858,0.216839,0.465659,5.197324
3,4,mean_squared_error,0:05:06.451410,97,0.189500,0.229295,0.216638,0.465444,5.207352
4,5,mean_squared_error,0:04:46.452250,92,0.192543,0.228036,0.215824,0.464569,5.226808
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:02:03.743139,50,0.191273,0.230419,0.216164,0.464934,5.230475
246,247,mean_squared_error,0:02:23.196040,58,0.192038,0.227559,0.215959,0.464714,5.246212
247,248,mean_squared_error,0:01:34.966605,38,0.191216,0.230462,0.216179,0.464951,5.216463
248,249,mean_squared_error,0:01:11.746546,29,0.184985,0.233981,0.217844,0.466738,5.217321


In [14]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.106363,0.208527,-0.107560,0.207330,-0.080738,0.234152,-0.092641,0.222249,-0.106148,...,-0.088585,0.226305,-0.128249,0.186641,-0.079163,0.235727,-0.079369,0.235521,-0.078189,0.236701
1,-0.267150,-0.335971,0.068821,-0.416756,0.149606,-0.308923,0.041773,-0.328997,0.061847,-0.291686,...,-0.261380,0.005770,-0.283757,0.016607,-0.262028,0.005122,-0.312943,0.045793,-0.260745,0.006405
2,0.257710,0.242710,0.015000,0.289920,0.032210,0.337488,0.079778,0.250313,0.007397,0.272763,...,0.325091,0.067381,0.252545,0.005165,0.331247,0.073537,0.394625,0.136915,0.320211,0.062501
3,-0.193330,-0.102681,0.090649,-0.100139,0.093191,-0.074237,0.119093,-0.085076,0.108254,-0.104071,...,-0.108305,0.085025,-0.124479,0.068851,-0.093893,0.099437,-0.094721,0.098609,-0.095860,0.097470
4,-0.409560,-0.135525,0.274035,-0.139184,0.270376,-0.109759,0.299801,-0.116209,0.293351,-0.128368,...,-0.115668,0.293892,-0.126447,0.283113,-0.109922,0.299638,-0.114046,0.295514,-0.108705,0.300855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.084329,0.063092,-0.089300,0.068063,-0.059869,0.038632,-0.068227,0.046990,-0.083066,...,-0.080517,0.059280,-0.095513,0.074276,-0.071891,0.050654,-0.069692,0.048455,-0.076898,0.055661
21447,-0.264320,0.053496,0.317816,0.085517,0.349837,0.110497,0.374817,0.090765,0.355085,0.080588,...,0.066078,0.330398,0.079077,0.343397,0.085814,0.350134,0.092008,0.356328,0.078314,0.342634
21448,0.060178,-0.109101,0.169279,-0.112426,0.172604,-0.084976,0.145154,-0.094101,0.154279,-0.106060,...,-0.089148,0.149326,-0.100420,0.160598,-0.079930,0.140108,-0.080294,0.140472,-0.078942,0.139120
21449,0.535320,0.217218,0.318102,0.263642,0.271678,0.310424,0.224896,0.223642,0.311678,0.245821,...,0.277712,0.257608,0.225980,0.309340,0.283333,0.251987,0.331273,0.204047,0.269961,0.265359


In [15]:
#Nifty call to confirm our variables were properly inputted into our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 16,897
Trainable params: 16,897
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:06:58.600213,133,0.186559,0.231756,0.217424,0.466287,5.227035
1,2,mean_squared_error,0:01:52.498483,36,0.181425,0.232965,0.218796,0.467756,5.223996
2,3,mean_squared_error,0:02:09.278538,41,0.188748,0.231858,0.216839,0.465659,5.197324
3,4,mean_squared_error,0:05:06.451410,97,0.189500,0.229295,0.216638,0.465444,5.207352
4,5,mean_squared_error,0:04:46.452250,92,0.192543,0.228036,0.215824,0.464569,5.226808
...,...,...,...,...,...,...,...,...,...
245,246,mean_squared_error,0:02:03.743139,50,0.191273,0.230419,0.216164,0.464934,5.230475
246,247,mean_squared_error,0:02:23.196040,58,0.192038,0.227559,0.215959,0.464714,5.246212
247,248,mean_squared_error,0:01:34.966605,38,0.191216,0.230462,0.216179,0.464951,5.216463
248,249,mean_squared_error,0:01:11.746546,29,0.184985,0.233981,0.217844,0.466738,5.217321


In [17]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4,m5,...,m246,res246,m247,res247,m248,res248,m249,res249,m250,res250
0,-0.314890,-0.106363,0.208527,-0.107560,0.207330,-0.080738,0.234152,-0.092641,0.222249,-0.106148,...,-0.088585,0.226305,-0.128249,0.186641,-0.079163,0.235727,-0.079369,0.235521,-0.078189,0.236701
1,-0.267150,-0.335971,0.068821,-0.416756,0.149606,-0.308923,0.041773,-0.328997,0.061847,-0.291686,...,-0.261380,0.005770,-0.283757,0.016607,-0.262028,0.005122,-0.312943,0.045793,-0.260745,0.006405
2,0.257710,0.242710,0.015000,0.289920,0.032210,0.337488,0.079778,0.250313,0.007397,0.272763,...,0.325091,0.067381,0.252545,0.005165,0.331247,0.073537,0.394625,0.136915,0.320211,0.062501
3,-0.193330,-0.102681,0.090649,-0.100139,0.093191,-0.074237,0.119093,-0.085076,0.108254,-0.104071,...,-0.108305,0.085025,-0.124479,0.068851,-0.093893,0.099437,-0.094721,0.098609,-0.095860,0.097470
4,-0.409560,-0.135525,0.274035,-0.139184,0.270376,-0.109759,0.299801,-0.116209,0.293351,-0.128368,...,-0.115668,0.293892,-0.126447,0.283113,-0.109922,0.299638,-0.114046,0.295514,-0.108705,0.300855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.084329,0.063092,-0.089300,0.068063,-0.059869,0.038632,-0.068227,0.046990,-0.083066,...,-0.080517,0.059280,-0.095513,0.074276,-0.071891,0.050654,-0.069692,0.048455,-0.076898,0.055661
21447,-0.264320,0.053496,0.317816,0.085517,0.349837,0.110497,0.374817,0.090765,0.355085,0.080588,...,0.066078,0.330398,0.079077,0.343397,0.085814,0.350134,0.092008,0.356328,0.078314,0.342634
21448,0.060178,-0.109101,0.169279,-0.112426,0.172604,-0.084976,0.145154,-0.094101,0.154279,-0.106060,...,-0.089148,0.149326,-0.100420,0.160598,-0.079930,0.140108,-0.080294,0.140472,-0.078942,0.139120
21449,0.535320,0.217218,0.318102,0.263642,0.271678,0.310424,0.224896,0.223642,0.311678,0.245821,...,0.277712,0.257608,0.225980,0.309340,0.283333,0.251987,0.331273,0.204047,0.269961,0.265359


In [18]:
model_record.to_csv('data/rnn_results_8_2.csv', index = False)
predict_record.to_csv('data/rnn_predictions_8_2.csv', index = False)